In [41]:
import pandas as pd
import glob

def load_salescate_data(data_folder):
    # Get the path to all files containing 'salescate' and ending with '.xlsx'
    salescate_files = glob.glob(data_folder + "/salebycategory*.xlsx")
    
    # If no files are found, raise a FileNotFoundError
    if not salescate_files:
        raise FileNotFoundError(f"No 'salebycategory*.xlsx' files found in {data_folder}")
    
    # Load all the salescate data into a list of pandas dataframes
    salescate_dfs = [pd.read_excel(file) for file in salescate_files]
    
    # Concatenate all the dataframes into one
    salescate_df = pd.concat(salescate_dfs, ignore_index=True)
    
    return salescate_df

# Define the data folder
data_folder1 = r'C:\Users\PC\Dropbox\Projects\data_xls\salebycategory'
data_folder2 = r'C:\Users\trieu.pham\Dropbox\Projects\data_xls\salebycategory'
data_folder3 = r'/Users/trieupham/Dropbox/Projects/data_xls/salebycategory'

try:
    df = load_salescate_data(data_folder1)
except FileNotFoundError:
    try:
        df = load_salescate_data(data_folder2)
    except FileNotFoundError:
        try:
            df = load_salescate_data(data_folder3)
        except FileNotFoundError:
            print("Data folder not found")
            raise
df.head()

,barcode,itemName,divisionCode,divisionName,categoryCode,categoryName,subCategoryCode,subCategoryName,itemQty,grossSales,itemDiscount,netSales,totalCostPrice
0,2010101000005,Ly đá vừa x ly,25,FF Beverage,2506,FF Beverage Onsite,250601,Cup Of Ice,666,1029000,969000,952845,599400.0
1,2501031190411,Ly đá lớn x ly,25,FF Beverage,2506,FF Beverage Onsite,250601,Cup Of Ice,541,2164000,0,2003799,762810.0
2,2501031190510,Trà tắc ly lớn x 1 ly,25,FF Beverage,2506,FF Beverage Onsite,250603,Cold Local Drinks,331,5730000,890000,5305625,3362794.5
3,8934588873553,TEA+ Trà Oolong 450ml x chai,7,Beverage,703,Tea,70301,Sugar Tea,216,2592000,0,2399978,1697760.0
4,8934588063053,AQUAFINA Nước tinh khiết 500ml x chai,7,Beverage,701,Bottled Water,70101,Purified water,208,1197000,259000,1108291,882700.0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22562 entries, 0 to 22561
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   barcode          22562 non-null  int64  
 1   itemName         22562 non-null  object 
 2   divisionCode     22562 non-null  int64  
 3   divisionName     22562 non-null  object 
 4   categoryCode     22562 non-null  int64  
 5   categoryName     22562 non-null  object 
 6   subCategoryCode  22562 non-null  int64  
 7   subCategoryName  22562 non-null  object 
 8   itemQty          22562 non-null  int64  
 9   grossSales       22562 non-null  int64  
 10  itemDiscount     22562 non-null  int64  
 11  netSales         22562 non-null  int64  
 12  totalCostPrice   22562 non-null  float64
dtypes: float64(1), int64(8), object(4)
memory usage: 2.2+ MB


In [43]:
# Load all df to FCdata.db table salesbycategory
import sqlite3


db_path_1 = r'C:\Users\trieu.pham\Dropbox\Projects\database\FCdata.db'
db_path_2 = r'C:\Users\PC\Dropbox\Projects\database\FCdata.db'
db_path_3 = '/Users/trieupham/Dropbox/Projects/database/FCdata.db' # Mac
db_path_4 = r'C:\Users\haitr\Dropbox\Projects\database\FCdata.db' # Alienware

import os
# Check if db_path exists
if os.path.exists(db_path_1):
    db_path = db_path_1
    print('Database path: ', db_path)
else:
    if(os.path.exists(db_path_3)):
        db_path = db_path_3
        print('Database path: ', db_path)
    if(os.path.exists(db_path_2)):
        db_path = db_path_2
        print('Database path: ', db_path)
    if(os.path.exists(db_path_4)):
        db_path = db_path_4
        print('Database path: ', db_path)

Database path:  C:\Users\PC\Dropbox\Projects\database\FCdata.db


In [44]:
# Check if table exists sales_by_category
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='sales_by_category'")
if cursor.fetchone() is None:
    print('Table sales_by_category does not exist')
else:
    print('Table sales_by_category exists')
    
conn.commit()
conn.close()

Table sales_by_category exists


In [45]:
# Load df to FCdata.db table sales_by_category
conn = sqlite3.connect(db_path)
df.to_sql('sales_by_category', conn, if_exists='replace', index=False)
conn.commit()
conn.close()


In [46]:
# Show the first 5 rows of the table sales_by_category and print results
conn = sqlite3.connect(db_path)
query = "SELECT * FROM sales_by_category LIMIT 5"
query2 = "SELECT COUNT(*) FROM sales_by_category"
df_query = pd.read_sql_query(query, conn)
df_query2 = pd.read_sql_query(query2, conn)
conn.close()
# df_query
df_query2


,COUNT(*)
0,22562


In [47]:
# Create new table item_category which have unique values based on barcode, it table should have columns: barcode	itemName	divisionCode	divisionName	categoryCode	categoryName	subCategoryCode	subCategoryName	
# data should be loaded from sales_by_category table
conn = sqlite3.connect(db_path)
# Check if table exists item_category
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='item_category'")
if cursor.fetchone() is None:
    print('Table item_category does not exist')
    query = """
    CREATE TABLE item_category AS 
    SELECT DISTINCT barcode, itemName, divisionCode, divisionName, categoryCode, categoryName, subCategoryCode, subCategoryName
    FROM sales_by_category
    """
    cursor = conn.cursor()
    cursor.execute(query)
else:
    print('Table item_category exists')
conn.commit()
conn.close()


Table item_category exists


In [48]:
# Check if any barcode exists in sales_by_category but not in item_category
conn = sqlite3.connect(db_path)
query = """
SELECT barcode, itemName
FROM sales_by_category
EXCEPT
SELECT barcode, itemName
FROM item_category
"""
df_query = pd.read_sql_query(query, conn)
conn.close()
df_query

,barcode,itemName


In [49]:
# for each barcode which exists in sales_by_category but not in item_category, insert a new row to item_category table, include barcode	itemName	divisionCode	divisionName	categoryCode	categoryName	subCategoryCode	subCategoryName	
conn = sqlite3.connect(db_path)
query = """
INSERT INTO item_category
SELECT DISTINCT barcode, itemName, divisionCode, divisionName, categoryCode, categoryName, subCategoryCode, subCategoryName
FROM sales_by_category
EXCEPT
SELECT barcode, itemName, divisionCode, divisionName, categoryCode, categoryName, subCategoryCode, subCategoryName
FROM item_category
"""
cursor = conn.cursor()
cursor.execute(query)
conn.commit()
conn.close()


In [50]:
# Query the first 5 rows of the table item_category and print results
conn = sqlite3.connect(db_path)
query = "SELECT * FROM item_category LIMIT 5"
df_query = pd.read_sql_query(query, conn)
df_query

,barcode,itemName,divisionCode,divisionName,categoryCode,categoryName,subCategoryCode,subCategoryName
0,2010101000005,Ly đá vừa x ly,25,FF Beverage,2506,FF Beverage Onsite,250601,Cup Of Ice
1,2501031190411,Ly đá lớn x ly,25,FF Beverage,2506,FF Beverage Onsite,250601,Cup Of Ice
2,2501031190510,Trà tắc ly lớn x 1 ly,25,FF Beverage,2506,FF Beverage Onsite,250603,Cold Local Drinks
3,8934588873553,TEA+ Trà Oolong 450ml x chai,7,Beverage,703,Tea,70301,Sugar Tea
4,8934588063053,AQUAFINA Nước tinh khiết 500ml x chai,7,Beverage,701,Bottled Water,70101,Purified water


In [51]:
# List out all barcodes have more than one row in dataframe, print out the barcode and the number of rows for each barcode, and itemName of each barcode
df['barcode'].value_counts()[df['barcode'].value_counts() > 1]


barcode
2010101000005    12
8936036027945    12
8936013680521    12
8938543817170    12
8691216018891    12
                 ..
8938532962706     2
8936087591532     2
8935136864924     2
8809711711883     2
8938539151004     2
Name: count, Length: 3063, dtype: int64

In [52]:
df_grouped = df.groupby(['barcode', 'itemName', 'divisionCode', 'divisionName', 'categoryCode', 'categoryName', 'subCategoryCode', 'subCategoryName']).agg({
    'itemQty': 'sum',
    'grossSales': 'sum',
    'itemDiscount': 'sum',
    'netSales': 'sum',
    'totalCostPrice': 'sum'
}).reset_index()

df_grouped

,barcode,itemName,divisionCode,divisionName,categoryCode,categoryName,subCategoryCode,subCategoryName,itemQty,grossSales,itemDiscount,netSales,totalCostPrice
0,21202010,HH Cam Úc 1 trái,29,Fresh Grocery,2901,Fruit Cat,290101,Fruit Sub,7,280000,0,266665,192500.0
1,23133289,GPS Thạch hồng sâm Innerset 15g x gói,7,Beverage,708,Others Beverage,70801,Condition/Supplements,53,1537000,0,1423151,939546.9
2,40677767,Jagermeister 200ml x Chai,14,Wine&Liquor,1401,Liquor,140102,Cordials/Brandy/Cognac,1,293000,0,266364,189000.0
3,40677798,Jagermeister 20ml x Chai,14,Wine&Liquor,1401,Liquor,140102,Cordials/Brandy/Cognac,47,2491000,0,2264554,1518100.0
4,45116568,Lotte Kem mochi Yukimi Daifuku 90ml x hộp,5,Ice Cream,501,Packaged Ice Cream/Novelties,50104,Other Ice Cream,296,9577000,231000,8867664,6458128.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4946,18935012413328,THÙNG 24SABECO Lager Bia 330ml x Lon,6,Beer,602,Domestic Beer,60201,Domestic Beer Can,5,1600000,0,1454545,1154180.0
4947,18935217400093,THÙNG48THTM STT Không Đường 220ml x Gói,2,Milk,201,Whole Milk,20102,UHT Milk,1,430000,0,398148,372144.0
4948,28935049502337,THÙNG 24_COCA Zero Sleek 320ml x 1 Lon,7,Beverage,702,Carbonated Drinks - CSD,70202,No/Less Sugar CSD,1,194000,66000,179630,199560.0
4949,48934588063068,THÙNG 12AQUAFINA Nước tinh khiết 1.5L x chai,7,Beverage,701,Bottled Water,70101,Purified water,1,140000,0,129630,104280.0


In [53]:
top_20_barcodes = df_grouped.sort_values('netSales', ascending=False).head(20)
top_20_barcodes

,barcode,itemName,divisionCode,divisionName,categoryCode,categoryName,subCategoryCode,subCategoryName,itemQty,grossSales,itemDiscount,netSales,totalCostPrice
429,2501031190510,Trà tắc ly lớn x 1 ly,25,FF Beverage,2506,FF Beverage Onsite,250603,Cold Local Drinks,2311,43550000,2670000,40324710,23478604.50
4388,8938502525368,Bánh bao nhân thịt heo trứng muối xá xíu,26,FF Onsite,2604,Steamed Bun,260402,Savoury Buns,993,28797000,0,26664027,18379799.45
369,2250103000058,TP. Bánh giò nhân thịt 150g x 1 cái,26,FF Onsite,2604,Steamed Bun,260403,Others Steamed bun,1995,27930000,0,25861185,15455124.36
2670,8934822112332,TIGER Bia lon cao 330ml x Lon,6,Beer,601,Import Beer,60101,Import Beer Can,1307,26187000,672000,23806363,17005847.52
318,2010805000363,Hotdog 25 Signature x cái,26,FF Onsite,2602,Other Cuisine,260201,Other Cuisine Sub,897,25100000,16000,23240808,17090693.49
283,2010103000010,Milo ly lớn x 1ly,25,FF Beverage,2506,FF Beverage Onsite,250602,Fountain-Non-carbonated,1002,23992000,1058000,22214742,14873402.92
4386,8938502525344,BB 2 trứng cút 150g x 1 Cái,26,FF Onsite,2604,Steamed Bun,260402,Savoury Buns,1202,23776000,0,22014865,14886714.83
573,2701010000179,GCCT Cơm nghêu mẹ nấu x hộp,27,FF Offsite,2701,Rice Offsite,270101,Bento,589,22382000,0,20723965,13646000.00
522,2602020019048,Tteokbokki Signature (TC) x tô,26,FF Onsite,2602,Other Cuisine,260202,Korean cuisine Sub,706,21232000,0,19659415,13707519.00
4387,8938502525351,BB muối 170g x 1 Cái,26,FF Onsite,2604,Steamed Bun,260402,Savoury Buns,882,21072000,0,19510928,14186592.78


In [54]:
# Filter item Milo 2010103000010
df[df['barcode'] == 2602020019048]

,barcode,itemName,divisionCode,divisionName,categoryCode,categoryName,subCategoryCode,subCategoryName,itemQty,grossSales,itemDiscount,netSales,totalCostPrice
5,2602020019048,Tteokbokki Signature (TC) x tô,26,FF Onsite,2602,Other Cuisine,260202,Korean cuisine Sub,205,6150000,0,5694489,3948126.0
3585,2602020019048,Tteokbokki Signature (TC) x tô,26,FF Onsite,2602,Other Cuisine,260202,Korean cuisine Sub,191,5782000,0,5353746,3741499.0
5507,2602020019048,Tteokbokki Signature (TC) x tô,26,FF Onsite,2602,Cuisine Cat,260202,Korean cuisine Sub,174,5561600,6400,5149687,3408486.0
7556,2602020019048,Tteokbokki Signature (TC) x tô,26,FF Onsite,2602,Other Cuisine,260202,Korean cuisine Sub,103,3090000,0,2861134,1966167.0
11020,2602020019048,Tteokbokki Signature (TC) x tô,26,FF Onsite,2602,Other Cuisine,260202,Korean cuisine Sub,207,6210000,0,5750046,4051727.0
12882,2602020019048,Tteokbokki Signature (TC) x tô,26,FF Onsite,2602,Cuisine Cat,260202,Korean cuisine Sub,145,4640000,0,4296345,2840405.0
18510,2602020019048,Tteokbokki Signature (TC) x tô,26,FF Onsite,2602,Cuisine Cat,260202,Korean cuisine Sub,172,5504000,0,5096352,3369308.0
20477,2602020019048,Tteokbokki Signature (TC) x tô,26,FF Onsite,2602,Cuisine Cat,260202,Korean cuisine Sub,145,4640000,0,4296345,2840405.0


In [55]:
df_summed = df.groupby('barcode').agg({
    'itemQty': 'sum',
    'grossSales': 'sum',
    'itemDiscount': 'sum',
    'netSales': 'sum',
    'totalCostPrice': 'sum'
}).reset_index()

df_summed

,barcode,itemQty,grossSales,itemDiscount,netSales,totalCostPrice
0,21202010,7,280000,0,266665,192500.0
1,23133289,53,1537000,0,1423151,939546.9
2,40677767,1,293000,0,266364,189000.0
3,40677798,47,2491000,0,2264554,1518100.0
4,45116568,296,9577000,231000,8867664,6458128.0
...,...,...,...,...,...,...
3368,18935012413328,5,1600000,0,1454545,1154180.0
3369,18935217400093,1,430000,0,398148,372144.0
3370,28935049502337,1,194000,66000,179630,199560.0
3371,48934588063068,1,140000,0,129630,104280.0


In [56]:
df_itemName = df.groupby('barcode')['itemName'].first()
df_summed['itemName'] = df_summed['barcode'].map(df_itemName)
df_summed

,barcode,itemQty,grossSales,itemDiscount,netSales,totalCostPrice,itemName
0,21202010,7,280000,0,266665,192500.0,HH Cam Úc 1 trái
1,23133289,53,1537000,0,1423151,939546.9,GPS Thạch hồng sâm Innerset 15g x gói
2,40677767,1,293000,0,266364,189000.0,Jagermeister 200ml x Chai
3,40677798,47,2491000,0,2264554,1518100.0,Jagermeister 20ml x Chai
4,45116568,296,9577000,231000,8867664,6458128.0,Lotte Kem mochi Yukimi Daifuku 90ml x hộp
...,...,...,...,...,...,...,...
3368,18935012413328,5,1600000,0,1454545,1154180.0,THÙNG 24SABECO Lager Bia 330ml x Lon
3369,18935217400093,1,430000,0,398148,372144.0,THÙNG48THTM STT Không Đường 220ml x Gói
3370,28935049502337,1,194000,66000,179630,199560.0,THÙNG 24_COCA Zero Sleek 320ml x 1 Lon
3371,48934588063068,1,140000,0,129630,104280.0,THÙNG 12AQUAFINA Nước tinh khiết 1.5L x chai


In [57]:

top_20_barcodes = df_summed.sort_values('netSales', ascending=False).head(20)
top_20_barcodes

,barcode,itemQty,grossSales,itemDiscount,netSales,totalCostPrice,itemName
285,2501031190510,3169,60706000,2674000,56210178,32196640.50,Trà tắc ly lớn x 1 ly
341,2602020019048,1342,41577600,6400,38498144,26166123.00,Tteokbokki Signature (TC) x tô
226,2010805000363,1485,41564000,16000,38485296,28253477.60,Hotdog 25 Signature x cái
210,2010103000010,1370,33192000,1058000,30733206,20233867.16,Milo ly lớn x 1ly
3003,8938502525368,993,28797000,0,26664027,18379799.45,Bánh bao nhân thịt heo trứng muối xá xíu
251,2250103000058,1995,27930000,0,25861185,15455124.36,TP. Bánh giò nhân thịt 150g x 1 cái
1790,8934822112332,1329,26649000,672000,24226365,17292097.44,TIGER Bia lon cao 330ml x Lon
81,260202000017,805,24150000,0,22361288,14575237.36,Tteokbokki Signature x 1 phần
3001,8938502525344,1202,23776000,0,22014865,14886714.83,BB 2 trứng cút 150g x 1 Cái
384,2701010000179,614,23332000,0,21603590,14221000.00,GCCT Cơm nghêu mẹ nấu x hộp
